# Lab 4 - Linear models - modelling height of !Kung people

During the excercise, we will be covering design of model for heights of certain population of people.
Specific tasks are in the laboratory notebook, following there are specifications for the required models. 

<hr>


In [1]:
from cmdstanpy import CmdStanModel


import arviz as az
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import pandas as pd

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Ex. 1 Prepare the models:

```height_1_ppc.stan```


Generated quantities model, that would sample $\mu$ from normal prior distribution. It is supposed to represent our general knowledge about heights of people so assign reasonable mean height and standard deviation of those values. Our measurement error is unknown, but reasonably it would be on average 15 cm, propose a distribution and parameters for prior of standard deviation $\sigma$. Finally sample measurement of ```height``` from normal distribution with previously sampled $\mu$ and $\sigma$. Use this model to select parameters for priors to provide reasonable spreads of heights. 

<hr>

```height_1_fit.stan```

Create a model for fitting model without predictors. 
For this purpose:

- specify ```data``` block with ```N``` samples, and input real array of size ```N``` called ```heights```
- specify ```parameters``` block with $\mu$ and $\sigma$ (set up appropriate types and constraints)
- in ```model``` block specify priors with appropriately selected parameters and normal likelihood using those parameters.
- in ```generated quantites``` block sample a single height consistent with the model.

<hr>

```height_2a_ppc.stan```

Modify the previous prior predictive check model by:
- adding ```data``` block with ```N``` samples, and input real array of size ```N``` called ```weight```
- new variable $\alpha$ for intercept with same prior as $\mu$ in previous model
- add a standard normal prior for predictor coefficient $\beta$
- create a size ```N``` real array called ```height``` and with a ```for``` loop fill it with normal samples with standard deviation $\sigma$ and mean $\alpha+\beta\cdot\mathrm{weight}[i]$

<hr>

```height_2b_ppc.stan```

Modify the previous prior predictive check model to use lognormal distribution for $\beta$.  

<hr>

```height_2_fit.stan```

Modify previous model for fitting, by:
- adding to ```data``` block an input real array of size ```N``` called ```weight```
- adding to ```parameters``` a new one - $\beta$
- create a new block ```transformed parameters``` where with a ```for``` loop a new real array parameter of size [N] is created with $\mu=\alpha+\beta\cdot\mathrm{weight}[i]$
- in ```model``` add the prior for $\beta$ and evaluate likelihood in a loop over all values of $\mu$ array.
- in ```generated quantites``` block sample ```N``` dimensional array height over all values of $\mu$ array.
We focus on models in the form
    $$y \sim \mathrm{Normal}(\alpha+X\beta,\sigma)$$

We will try to fit $\mathrm{Normal}(\mu, \sigma)$ distribution to height data. Special case of linear model with $\beta=0$.

In [2]:
_BASE_URL = "https://raw.githubusercontent.com/rmcelreath/rethinking/Experimental/data"
HOWELL_DATASET_PATH = f"{_BASE_URL}/Howell1.csv"
d = pd.read_csv(HOWELL_DATASET_PATH, sep=';', header=0)
d=d[d.age>=18] #just adults 
d.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0


#### Task 1. Prior predictive check

In [4]:
model_ppc=CmdStanModel(stan_file='height_1_ppc.stan')

R = 1000
data = {"mu":d.height.mean(), "sigma":d.height.std()}
sim=model_ppc.sample(data=data,
                     iter_sampling=R,
                     iter_warmup=0,
                     chains=1,
                     seed=29042020,refresh=R)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed                      


INFO:cmdstanpy:CmdStan done processing.


In [6]:
sim_df = sim.draws_pd()
sim_df

,lp__,accept_stat__,mu_p,sigma_p,height
0,0.0,0.0,0.815985,2.45843,0.000000
1,0.0,0.0,0.815985,2.45843,0.659479
2,0.0,0.0,0.815985,2.45843,0.000000
3,0.0,0.0,0.815985,2.45843,1.129660
4,0.0,0.0,0.815985,2.45843,0.000000
...,...,...,...,...,...
995,0.0,0.0,0.815985,2.45843,2.454840
996,0.0,0.0,0.815985,2.45843,2.875930
997,0.0,0.0,0.815985,2.45843,0.086271
998,0.0,0.0,0.815985,2.45843,0.000000
